In [ ]:
import sys,os,re
import pysubs2
import cv2
import threading

## 文字修补

In [ ]:
def Inpainting(srcImg,x,y,xx,yy,style="文本",kernelSize=7,iter=1,r=3):
    src = srcImg[x:xx,y:yy].copy()
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY,1)
    if "fadeout" in style:
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    elif style == "学生" or style == "文本": #白字
        thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)[1]
    else:
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))
    maskImg = cv2.dilate(thresh, kernel,iterations=iter)
    inpaintImg = cv2.inpaint(src,maskImg,r,cv2.INPAINT_NS)
    srcImg[x:xx,y:yy] = inpaintImg

## 读取 config

In [ ]:
axis = {}
with open("config.txt", "r",encoding='utf-8') as f:
    for line in f.readlines():
        line = line.strip('\n')
        line = line.split(':')
        style = line[0]
        axis[style] = list(map(int,list(line[1].split(','))))

print(axis)

{'标题': [400, 580, 1240, 675], '羁绊标题': [535, 540, 1054, 632], '章节': [705, 505, 885, 570], '学生': [105, 900, 570, 966], '文本': [114, 988, 1500, 1154], '单选': [435, 495, 1200, 550], '选项1': [604, 435, 1029, 501], '选项2': [595, 560, 1029, 615]}


In [ ]:
#测试
for i in range(length):
    ret, img = cap.read() # 进度
    print(i/length*100)
    if ret:
        for style in frame[i]:
            y1 = axis[style][0]
            x1 = axis[style][1]
            x2 = axis[style][3]
            y2 = axis[style][2]
            Inpainting(img,x1,y1,x2,y2,style)
        out.write(img)
        #img = cv2.resize(img, (0, 0),fx = 0.5,fy = 0.5)
        #cv2.imshow('img',img)
    else:
        print("Error")
        break
    if cv2.waitKey(2) & 0xFF==27:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

## 时轴预处理

拆分「人物（社团）：文本」

文本换行

In [ ]:
filename = "小夏4"
path = sys.path[0]+"\\temp\\"
cap = cv2.VideoCapture("小夏4.mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
width = int(cap.get(3))
height = int(cap.get(4))
frame = []
for i in range(length):
    frame.append([])
if not os.path.exists(path):
    os.makedirs(path)

#filename_out.ass
subs = pysubs2.load(filename+".ass")
for sub in subs:
    start = pysubs2.time.ms_to_frames(sub.start, fps)
    end = pysubs2.time.ms_to_frames(sub.end, fps)

    if sub.style == "文本":
        name_school = sub.text.split("：")[0]
        text = sub.text.split("：")[-1]

        if name_school != text:
            name = name_school.split('（')[0]
            school = name_school.split('（')[-1].replace("）","")
            subs.insert(0, pysubs2.SSAEvent(start=sub.start, end=sub.end, text=name+" {\\fs35 \c&H f4ca80 }"+school, style="学生"))

        if len(text) > 35: # 换行
            text = "\\N".join(re.findall(".{35}",text)) 
        sub.text = text

        for i in range(start,end):
            frame[i].append("学生")
            frame[i].append("文本")

    else:
        for i in range(start,end):
            frame[i].append(sub.style)
subs.save(path + filename + "_out.ass")

## 多线程处理

In [ ]:
# 多线程-任务
def work(start):
    video_path = os.path.join("./temp/", "part_"+str(start)+".mp4")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(3))
    height = int(cap.get(4))
    out = cv2.VideoWriter("./temp/part_"+str(start)+"_out.mp4",fourcc,fps,(width,height))
    print("start "+str(start))

    for i in range(start*num,(start+1)*num):
        ret, img = cap.read()
        if ret:
            for style in frame[i]:
                y1 ,x1 ,y2 ,x2= axis[style][0], axis[style][1], axis[style][2], axis[style][3]
                Inpainting(img,x1,y1,x2,y2,style,kernelSize=9,iter=1,r=4)
            out.write(img)
            # img = cv2.resize(img, (0, 0),fx = 0.5,fy = 0.5)
            # cv2.imshow('img',img)
        else:
            break
        if cv2.waitKey(1) & 0xFF==27:
            break
    cap.release()
    out.release()
    # cv2.destroyAllWindows()

In [ ]:
# 多线程-切分
num = 1000
i = -1

with open(path + "list.txt","a") as f:
    while 1:
        i += 1
        ret, img = cap.read()
        if ret:
            if i % num == 0:
                out = cv2.VideoWriter(path + "part_" + str(int(i/num)) + ".mp4",fourcc,fps,(width,height))
                f.write(f"file ./temp/part_{id}_out.mp4\n".format(id=int(i/num)))
            out.write(img)
            # img = cv2.resize(img, (0, 0),fx = 0.5,fy = 0.5)
            # cv2.imshow('img',img)
            #print(i)
        else:
            break
        if cv2.waitKey(1) & 0xFF==27:
            break
    cap.release()
    out.release()
    # cv2.destroyAllWindows()

In [ ]:
thread_list = []

for i in range(0,8):

    thread = threading.Thread(target=work, args=[i])
    thread.start()
    thread_list.append(thread)

for t in thread_list:

    t.join()

## 导出

In [ ]:
os.system("ffmpeg -f concat -safe 0 -i {path}list.txt -c copy {path}concat.mp4".format(path=path))
os.system("ffmpeg -i {path}concat.mp4 -i {filename}.mp4  -c copy -map 0 -map 1:1 -y -shortest {path}{filename}_out.mp4".format(path=path,filename="小夏4"))
os.system("ffmpeg -i {path}{filename}_out.mp4 -vf subtitles=temp/{filename}_out.ass {filename}_final.mp4".format(path=path,filename="小夏4"))
#os.system("rd/s/q "+path)

# 音频
os.system("ffmpeg -i {filename}_temp.mp4 -i {filename}.mp4  -c copy -map 0 -map 1:1 -y -shortest {filename}_out.mp4".format(filename="小夏4"))
os.system("del {filename}_temp.mp4".format(filename="小夏4"))

# 字幕https://zhuanlan.zhihu.com/p/161000532
os.system("ffmpeg -i {path}{filename}_out.mp4 -vf subtitles=temp/{filename}_out.ass {filename}_final.mp4".format(path=path,filename="小夏4"))
os.system("del {filename}_out.mp4".format(filename="小夏4"))
os.system("del {filename}_out.ass".format(filename="小夏4"))

## 打印机效果

每个字间隔两帧🎞，大约 33ms
{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(33,34,\1a&H00&\3a&H00&\4a&H00&)}

In [ ]:
subs = pysubs2.load("小夏4.ass")

for i in range(5):
    if subs[i].style == "文本":
        text = subs[i].text.split("：")[-1]
        print(text)

for i in range(5):
    if subs[i].style == "文本":
        text = sub.text.split("：")[-1]
        cnt = 1
        s = ""
        for word in text:
            s = s+"{\\1a&HFF&\\3a&HFF&\\4a&HFF&}{\\t("+str(cnt*33)+","+str(cnt*33+1)+",\\1a&H00&\\3a&H00&\\4a&H00&)}"+word
            if cnt % 35 == 0:
                s += "\\N"
            cnt+=1
        print(s)